# 1.

In [1]:
%pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2.

In [2]:
mp_holistic = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

In [6]:
def extract_keypoints(results):
    try:
        for idx, handLms in enumerate(results.multi_hand_landmarks):
            lh = np.array([[res.x, res.y, res.z] for res in
                           results.multi_hand_landmarks[idx].landmark]).flatten() \
                if results.multi_handedness[idx].classification[0].label == 'Left' else np.zeros(21 * 3)
            rh = np.array([[res.x, res.y, res.z] for res in
                           results.multi_hand_landmarks[idx].landmark]).flatten() \
                if results.multi_handedness[idx].classification[0].label == 'Right' else np.zeros(21 * 3)
        return np.concatenate([lh, rh])
    except:
        return np.concatenate([np.zeros(21 * 3), np.zeros(21 * 3)])

# 3.

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
actions = np.array([ 'up', 'down', 'left', 'right', 'ok', 'back'])

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'up': 0, 'down': 1, 'left': 2, 'right': 3, 'ok': 4, 'back': 5}

In [11]:
sequences = np.load("sequences.npy")
labels = np.load("labels.npy")

In [12]:
def interp_coords(x):
    coords = []
    for num,i in enumerate(x):
        if np.count_nonzero(i) != 0:
            coords.append([i,num])
    if not coords:
        return x
    result = []
    for i in range(126):
        result.append(np.interp(range(40),[e[1] for e in coords],[e[0][i] for e in coords]))
    return np.array(result).transpose()

In [13]:
sequences = np.array([interp_coords(e) for e in sequences])

In [14]:
np.array(sequences).shape

(180, 40, 126)

In [15]:
sequences[117,:,13]

array([0.72091711, 0.5262872 , 0.5289821 , 0.56253177, 0.5457027 ,
       0.538248  , 0.5415135 , 0.54266447, 0.55735153, 0.55243224,
       0.62361568, 0.62492919, 0.62459338, 0.62900215, 0.6250667 ,
       0.63076842, 0.57194453, 0.64411694, 0.63277876, 0.63513589,
       0.64984399, 0.66455209, 0.67294908, 0.71240586, 0.72480088,
       0.72308987, 0.72365177, 0.726358  , 0.7267794 , 0.72064334,
       0.72772491, 0.72834146, 0.72573471, 0.72409016, 0.71889502,
       0.72127241, 0.72346354, 0.72411078, 0.72424698, 0.72339386])

In [16]:
np.array(labels).shape

(180,)

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(180, 40, 126)

In [19]:
y = to_categorical(labels).astype(int)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [22]:
y_test.shape

(18, 6)

# 4.

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(40,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
6/6 [==============================] - 8s 109ms/step - loss: 1.7829 - categorical_accuracy: 0.1728
Epoch 2/100
6/6 [==============================] - 1s 122ms/step - loss: 1.7460 - categorical_accuracy: 0.2778
Epoch 3/100
6/6 [==============================] - 1s 116ms/step - loss: 1.6570 - categorical_accuracy: 0.2407
Epoch 4/100
6/6 [==============================] - 1s 111ms/step - loss: 1.5603 - categorical_accuracy: 0.2963
Epoch 5/100
6/6 [==============================] - 1s 109ms/step - loss: 1.5202 - categorical_accuracy: 0.3148
Epoch 6/100
6/6 [==============================] - 1s 107ms/step - loss: 1.9403 - categorical_accuracy: 0.4136
Epoch 7/100
6/6 [==============================] - 1s 109ms/step - loss: 1.6135 - categorical_accuracy: 0.3889
Epoch 8/100
6/6 [==============================] - 1s 109ms/step - loss: 1.6216 - categorical_accuracy: 0.3395
Epoch 9/100
6/6 [==============================] - 1s 94ms/step - loss: 1.5447 - categorical_accuracy: 0.3210
Ep

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 40, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 203,558
Trainable params: 203,558
Non-trai

# 5.

In [29]:
res = model.predict(X_test)

In [30]:
actions[np.argmax(res[4])]

'down'

In [31]:
actions[np.argmax(y_test[4])]

'down'

# 6. 

In [33]:
model.save('action.h5')

In [32]:
del model

In [24]:
model.load_weights('action.h5')

# 7. 

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, recall_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[14,  0],
        [ 0,  4]],

       [[15,  0],
        [ 0,  3]],

       [[15,  0],
        [ 1,  2]],

       [[15,  1],
        [ 0,  2]],

       [[15,  0],
        [ 1,  2]],

       [[14,  1],
        [ 0,  3]]], dtype=int64)

In [36]:
accuracy_score(ytrue, yhat)

0.8888888888888888

# 8.

In [39]:
from scipy import stats

In [40]:
colors = [(245,117,16), (117,245,16), (16,117,245), (16,217,245), (116,117,245), (116,217,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [41]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-40:]
        
        if len(sequence) == 40:
            res = model.predict(np.expand_dims(interp_coords(sequence), axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti